# ROSDev 2019

### Tips on (with no specific order):

1. How to setup different robots in simulation using ros-controls
2. How to use and create custom gazebo_ros plugins
3. How to optimize the simulation from the model point of view
4. Namespaces and launch files

### Typical use cases of a simulation

- Safety check.. i.e. test whether any change in your algorithms have some problems without breaking anything
- Quality check.. i.e. whether any change in your codebase has some effect on the production cycle time, or the battery misuse.. it's a pity that Luca talked about PCl filters and not functionality tests..
- Generating data to feed machine learning knowledge
- Aid the mechanical design.. I got a master student that used gazebo to test whether his idea of a car with som helix would work at all before even building it
- Develop algorithms in remote, without having the need
- And nowadays, as a support for selling.. if you see two ads of similar robots, where the motion of one are like faked, you konw, like bouncing at a different acceleration than gravity, and the other one is done through a simulation.. don't you think you would trust more to the one that is more phisycally plausible?

# Chess Lab

A chess board with simplified pieces marked with aruco markers, two UR3 arms equipped with two robotiq grippers, and two realsense cameras. The combination of the robotic components are very common in universities and research center. The use of markers allows to have a simple but comprehensive sense-plan-actuate example with external sync (one player must wait for the other to finish the play to move), for educational or challenge purposes.

Authors: Jan Rosell and Carlos J. Rosales

![sim](images/sim.png)![real](images/real.jpg)

## Init and Test

The `main` package is called `chesslab_setup`. Within you will find configuration, launch, and model files. Mostly integrating files from other packages.

There is a `dashboard.launch.xml` that contains all modules that are to be used. The `xml` extension is just to avoid the autocompletion in the terminal. You might think of it as a library of things you can do. Almost everything is turned OFF by default.

So let's try for instance to load the cell without cameras by typing this on a terminal:

``` sh
roslaunch chesslab_setup no_cameras_sim.launch
```

REMINDER: To use it on the real scenario, you need to follow the steps in the continuous integration file  found at `~/catkin_ws/src/ros1819-final-project/bitbucket-pipelines.yml`, and also system configuration steps for the UR arm, robotiq gripper and cameras you will find in their own packages.

Now, let's try to move both robots to test everything works:

In [ ]:
import rospy

import actionlib

from control_msgs.msg import (
    FollowJointTrajectoryAction,
    FollowJointTrajectoryGoal,
)
from trajectory_msgs.msg import (
    JointTrajectoryPoint,
)


rospy.init_node("trajectory_client")


action_name_A="/team_A_arm/joint_trajectory_controller/follow_joint_trajectory"

client_A = actionlib.SimpleActionClient(action_name_A, FollowJointTrajectoryAction)
client_A.wait_for_server(timeout=rospy.Duration(10.0))

joints_A = ['team_A_shoulder_pan_joint','team_A_shoulder_lift_joint','team_A_elbow_joint','team_A_wrist_1_joint','team_A_wrist_2_joint','team_A_wrist_3_joint']


action_name_B="/team_B_arm/joint_trajectory_controller/follow_joint_trajectory"

client_B = actionlib.SimpleActionClient(action_name_B, FollowJointTrajectoryAction)
client_B.wait_for_server(timeout=rospy.Duration(10.0))

joints_B =['team_B_shoulder_pan_joint','team_B_shoulder_lift_joint','team_B_elbow_joint','team_B_wrist_1_joint','team_B_wrist_2_joint','team_B_wrist_3_joint']


point1 = JointTrajectoryPoint()
point1.positions = [-1.3,0.0,-1.8,0.3,0.1,0]
point1.time_from_start = rospy.Duration(2.0)

point2 = JointTrajectoryPoint()
point2.positions = [-1.3,-0.3,-1.8,0.3,0.7,0]
point2.time_from_start = rospy.Duration(5.0)


goal_A = FollowJointTrajectoryGoal()
goal_A.trajectory.joint_names = joints_A
goal_A.trajectory.header.stamp = rospy.Time.now()+rospy.Duration(0.1)
goal_A.trajectory.points.append(point1)
goal_A.trajectory.points.append(point2)

goal_B = FollowJointTrajectoryGoal()
goal_B.trajectory.joint_names = joints_B
goal_B.trajectory.header.stamp = rospy.Time.now()+rospy.Duration(0.1)
goal_B.trajectory.points.append(point1)
goal_B.trajectory.points.append(point2)

client_A.send_goal(goal_A)
client_B.send_goal(goal_B)

client_A.wait_for_result()
client_B.wait_for_result()

print "Trajectoy finished"

## Gazebo Ros Control

This is a __Model__ plugin that creates the hardware interface in a generic way for any robot with actuated joints.

That means that, if one has an arm and a gripper (very common use case), the plugin needs to be added _per model_.

For instance, this what the URDF file looks for the arm:

```xml
<robot xmlns:xacro="http://ros.org/wiki/xacro" name="team_A_arm">

	<xacro:include filename="$(find ur_description)/urdf/ur3.urdf.xacro" />

	<link name="world" />

	<joint name="team_A_calibration_joint" type="fixed">
		<parent link="world" />
		<child link="team_A_base_link" />
		<origin xyz="0.37 0.0 -0.04" rpy="0.0 0.0 1.5708" />
	</joint>
	<xacro:ur3_robot prefix="team_A_" joint_limited="true" precise_collision="false" />

	<gazebo>
		<plugin name="team_A_arm_hwiface" filename="libgazebo_ros_control.so">
			<robotNamespace>team_A_arm</robotNamespace>
		</plugin>
	</gazebo>
</robot>

```

NOTE: we have a modified version of the `ur_description` package from ROS-I, since it adds the gazebo ros control plugin by default, and can't be customized.

And this is what the gripper looks like:


```xml
<robot xmlns:xacro="http://ros.org/wiki/xacro" name="team_A_gripper">

	<xacro:include filename="$(find robotiq_2f_model)/model/robotiq_2f_85.urdf.xacro" />
	<xacro:include filename="$(find robotiq_2f_model)/model/robotiq_ur_coupler.urdf.xacro" />

	<link name="team_A_tool0" />

	<xacro:robotiq_ur_coupler name="team_A_coupler_link" parent="team_A_tool0" />

	<xacro:robotiq_2f_85 name="team_A_gripper" with_pads="true" parent="team_A_coupler_link" precise_collision="false" adaptive_transmission="false">
		<origin xyz="0 0 0" rpy="0 0 ${0.5*pi}" />
	</xacro:robotiq_2f_85>

	<gazebo>
		<plugin name="team_A_gripper_hwiface" filename="libgazebo_ros_control.so">
			<robotNamespace>team_A_gripper</robotNamespace>
		</plugin>
	</gazebo>
```

Both are in different files, and _spawned_ as separated models in gazebo. Since each one has its own `gazebo_ros_control` plugin, all topics and resources are particular for the model.

Note that, the root link of the arm is called `world`, which makes it be static in gazebo by default. And that is ok for now. In the gripper, we don't want it to be statc, but _screwed_ to the last link of the arm, and move when any joint is moved. We set the root link as the last link of the arm it is supposed to be attached to. This is just to work with the `robot_state_publisher`, so using the same name, we _fool_ `/tf` to have a unique tree. However, in gazebo, the gripper will fall to the floor. So we need a plugin to _screw_ it to the top of the last link.

## Custom plugin: gazebo_ros_always_attached

This is a custom plugin (inspired by the work of another) that allows to configure a joint between two models at two specified links.

Just want to highlight that when writing gazebo plugins, it is important to think what we are going to simulate with this. 

In our case, since we are _screwing_ two models, this can't be Model plugin, but a __World__ plugin, as it needs to be aware of the two models to be attached.

One particular thing in this case is that, as it would happen in real life, you need to specify where to screw the part, and from there, continue simulating.

Consistently, you will find this plugin in the world file found at `~/catkin_ws/src/ros1819-final-project/chesslab_setup/worlds/chesslab.world`, for example, like:

```xml
<plugin name="team_A_arm_gripper_joint" filename="libgazebo_ros_always_attached.so">
    <model_1>team_A_arm</model_1>
    <link_1>team_A_wrist_3_link</link_1>
    <model_2>team_A_gripper</model_2>
    <link_2>team_A_tool0</link_2>
    <link2_world_pose_x>0.174</link2_world_pose_x>
    <link2_world_pose_y>0.457</link2_world_pose_y>
    <link2_world_pose_z>0.027</link2_world_pose_z>
    <link2_world_pose_qx>0.500</link2_world_pose_qx>
    <link2_world_pose_qy>-0.500</link2_world_pose_qy>
    <link2_world_pose_qz>-0.500</link2_world_pose_qz>
    <link2_world_pose_qw>0.500</link2_world_pose_qw>
</plugin>
```

The pose corresponds to the forward kinematics of the robot at the spawned configuration. If you change it, then you must update the _screwing_ pose of the gripper.. obviusly.

## Collisions

Simplify a model also consider the collision geometry. It is a very common __lazy__ practice to use the same meshes coming from CAD programs in the visual as well in the collision.

Simplifying this geometry has implications in simulation and also in motion planning, where collision check consumes aroun 80% of the time. Again, the `ur_description` package is modified to add the `precise_collision` parameter as seen above.

Here is what the collision geometry of arm and gripper looks like with respect to the visual:

![visual_collision](images/visual_collision.png)

## Launches and Namespaces

__ROS__, as in any programming environment, allows to do the same thing in several way. One may argue pros and cons, but IMHO, I think it is good overall.

Through several projects, I have developed a way to write launch files and exploit _namespaces_. But no matter how many launch files you have written, a new project always add a new feature.

In this project, we wanted to be as modular as possible. Thus, as mentioned before, there is a single launch file containing all modules to be used, and include it with different combination of parameters in handy launch files.


Another thing was to avoid the _holy_ `/robot_description` parameter in global namespace, as each URDF corrsponds to each piece of hardware... but _namespaced_ for each module. So you will see through the `dashboard.launch.xml`, different `robot_description` parameters, with its own holy pair of `joint_state_publisher` and `robot_state_publisher`. For instace, for one of arm, the corresponding part might look like:

```xml
<group if="$(arg team_A_enabled)" ns="team_A_arm">

    <param name="robot_description" command="$(find xacro)/xacro $(find chesslab_setup)/robot/team_A_arm.urdf.xacro" />

    <node name="joint_state_publisher" pkg="joint_state_publisher" type="joint_state_publisher">
        <param name="use_gui" value="$(arg joint_gui)"/>
    </node>

    <param name="publish_frequency" value="100"/>
	<node name="robot_state_publisher" pkg="robot_state_publisher" type="robot_state_publisher" />

	<!-- arm controllers -->
	<node name="load_and_start_controller" pkg="controller_manager" type="spawner" respawn="false" output="screen" args="joint_state_controller joint_trajectory_controller" />


	<!-- arm hwiface -->
	<group if="$(arg load_real)">
		<node name="ur_hardware_interface" pkg="ur_modern_driver" type="ur_driver" output="screen">
			<!-- here there is a set of params to work with the real robot -->
		</node>
	</group>

    <node name="team_A_arm_gui" pkg="rqt_gui" type="rqt_gui" respawn="false" args="perspective-file $(find chesslab_setup)/config/team_A.perspective" output="screen" />
</group>
```

The last part is interesting. I wanted to try the `joint_trajectory_controller` rqt plugin. But it loads by default any URDF in the `robot_description` parameter. And AFIK, there is no way to change it (at first, I thought of loading each URDF in a parameter called `team_A_arm_description`, `team_A_gripper_description`.. and so on). So in this case, I relied on namespacing. And this way, it works correctly.

## Machine Virtualization

Basically, if you want to avoid installing simulation-wise dependencies, options are (mostly):

* 2 PC, one set with all dependencies.. which costs money and time
* 1 PC, one with a docker image running gazebo.. which costs plenty of time if you don't know docker
* 1 PC + ROSDs, where I hope you enjoy as much as I did preparing this ROSject ;)


# Thanks.